In [1]:
import sys
sys.path.append('../..')
import pennylane.numpy as np
import matplotlib.pyplot as plt
import pennylane as qml

In [2]:
from qubap.pennylane.costfunc_barren_plateau import global2local, test_hamiltonian
from qubap.pennylane.variational_algorithms import VQE

In [3]:
num_wires = 6
hamiltonian = test_hamiltonian( num_wires )
hamiltonian

<Hamiltonian: terms=64, wires=[0, 1, 2, 3, 4, 5]>


In [4]:
hamiltonian_local = global2local( hamiltonian )
hamiltonian_local

  (-0.08072916666666667) [Z4]
+ (-0.08072916666666667) [Z5]
+ (-0.08072916666666667) [Z3]
+ (-0.08072916666666667) [Z2]
+ (-0.08072916666666667) [Z1]
+ (-0.08072916666666667) [Z0]
+ (0.48437500000000017) [I0]


In [7]:
def ansatz(params, num_layers=2):
    
    num_wires = len(params)//(2*num_layers+2)

    K = 0
    for i in range(num_layers):
        qml.broadcast( qml.RY, wires=range(num_wires), pattern='single', parameters=params[K:K+num_wires] )
        qml.broadcast( qml.RZ, wires=range(num_wires), pattern='single', parameters=params[K+num_wires:K+2*num_wires] )
        qml.broadcast( qml.CNOT, wires=range(num_wires), pattern='ring')
        K = K+2*num_wires

    qml.broadcast( qml.RY, wires=range(num_wires), pattern='single', parameters=params[K:K+num_wires] )
    qml.broadcast( qml.RZ, wires=range(num_wires), pattern='single', parameters=params[K+num_wires:K+2*num_wires] )

In [11]:
optimizer = qml.SPSAOptimizer(maxiter=100)
maxiter   = 100

In [17]:
params_in = np.random.randn( 6*num_wires )*np.pi
params_out, energies_global = VQE( hamiltonian, ansatz, params_in, optimizer, shots=2**6, max_iterations=maxiter )
params_out, energies_local  = VQE( hamiltonian_local, ansatz, params_in, optimizer, shots=2**6, max_iterations=maxiter )
params_out, energies_global_after_local = VQE( hamiltonian_local, ansatz, params_out[-1], optimizer, shots=2**6, max_iterations=maxiter )

KeyboardInterrupt: 

In [ ]:
plt.plot( np.mean( energies_global, axis=0), 'o' )
plt.plot( np.mean( energies_local, axis=0), 'o' )
plt.plot( np.mean( energies_global_after_local, axis=0), 'o' )
plt.legend([ 'global', 'local', 'global after local'])

NameError: name 'energies_global' is not defined